In [5]:
codenames_words = ["africa", "agent", "air", "alien", "alps", "amazon", "ambulance", "america", "angel", "antarctica", "apple", "arm", "atlantis", "australia", "aztec", "back", "ball", "band", "bank", "bar", "bark", "bat", "battery", "beach", "bear", "beat", "bed", "beijing", "bell", "belt", "berlin", "bermuda", "berry", "bill", "block", "board", "bolt", "bomb", "bond", "boom", "boot", "bottle", "bow", "box", "bridge", "brush", "buck", "buffalo", "bug", "bugle", "button", "calf", "canada", "cap", "capital", "car", "card", "carrot", "casino", "cast", "cat", "cell", "centaur", "center", "chair", "change", "charge", "check", "chest", "chick", "china", "chocolate", "church", "circle", "cliff", "cloak", "club", "code", "cold", "comic", "compound", "concert", "conductor", "contract", "cook", "copper", "cotton", "court", "cover", "crane", "crash", "cricket", "cross", "crown", "cycle", "czech", "dance", "date", "day", "death", "deck", "degree", "diamond", "dice", "dinosaur", "disease", "doctor", "dog", "draft", "dragon", "dress", "drill", "drop", "duck", "dwarf", "eagle", "egypt", "embassy", "engine", "england", "europe", "eye", "face", "fair", "fall", "fan", "fence", "field", "fighter", "figure", "file", "film", "fire", "fish", "flute", "fly", "foot", "force", "forest", "fork", "france", "game", "gas", "genius", "germany", "ghost", "giant", "glass", "glove", "gold", "grace", "grass", "greece", "green", "ground", "ham", "hand", "hawk", "head", "heart", "helicopter", "himalayas", "hole", "hollywood", "honey", "hood", "hook", "horn", "horse", "horseshoe", "hospital", "hotel", "ice", "icecream", "india", "iron", "ivory", "jack", "jam", "jet", "jupiter", "kangaroo", "ketchup", "key", "kid", "king", "kiwi", "knife", "knight", "lab", "lap", "laser", "lawyer", "lead", "lemon", "leprechaun", "life", "light", "limousine", "line", "link", "lion", "litter", "loch ness", "lock", "log", "london", "luck", "mail", "mammoth", "maple", "marble", "march", "mass", "match", "mercury", "mexico", "microscope", "millionaire", "mine", "mint", "missile", "model", "mole", "moon", "moscow", "mount", "mouse", "mouth", "mug", "nail", "needle", "net", "new york", "night", "ninja", "note", "novel", "nurse", "nut", "octopus", "oil", "olive", "olympus", "opera", "orange", "organ", "palm", "pan", "pants", "paper", "parachute", "park", "part", "pass", "paste", "penguin", "phoenix", "piano", "pie", "pilot", "pin", "pipe", "pirate", "pistol", "pit", "pitch", "plane", "plastic", "plate", "platypus", "play", "plot", "point", "poison", "pole", "police", "pool", "port", "post", "pound", "press", "princess", "pumpkin", "pupil", "pyramid", "queen", "rabbit", "racket", "ray", "revolution", "ring", "robin", "robot", "rock", "rome", "root", "rose", "roulette", "round", "row", "ruler", "satellite", "saturn", "scale", "school", "scientist", "scorpion", "screen", "scuba diver", "seal", "server", "shadow", "shakespeare", "shark", "ship", "shoe", "shop", "shot", "sink", "skyscraper", "slip", "slug", "smuggler", "snow", "snowman", "sock", "soldier", "soul", "sound", "space", "spell", "spider", "spike", "spine", "spot", "spring", "spy", "square", "stadium", "staff", "star", "state", "stick", "stock", "straw", "stream", "strike", "string", "sub", "suit", "superhero", "swing", "switch", "table", "tablet", "tag", "tail", "tap", "teacher", "telescope", "temple", "theater", "thief", "thumb", "tick", "tie", "time", "tokyo", "tooth", "torch", "tower", "track", "train", "triangle", "trip", "trunk", "tube", "turkey", "undertaker", "unicorn", "vacuum", "van", "vet", "wake", "wall", "war", "washer", "washington", "watch", "water", "wave", "web", "well", "whale", "whip", "wind", "witch", "worm", "yard"];

In [6]:
import itertools
import numpy as np
from sklearn.decomposition import PCA


word = []
vec = []

max_words = 30000

with open("/Users/szymon/Downloads/google-10000-english-usa.txt") as f:
    common_words = f.read().strip().split('\n')
    
needed = set(common_words + codenames_words)
with open("/Users/szymon/Downloads/glove.42B.300d.txt") as f:
    for word_idx, line in enumerate(itertools.islice(f, 100000)):
        w, v = line.strip().split(' ', 1)
        v = np.array(list(map(float, v.split(' '))), dtype=np.float32)
        if not w.isalpha() or not w.isascii() or (word_idx >= max_words and w not in needed):
            continue
        word.append(w)
        vec.append(v)
        
# hacks
vec.append((vec[word.index('new')] + vec[word.index('york')]) / 2)
word.append('new york')
vec.append((vec[word.index('loch')] + vec[word.index('ness')]) / 2)
word.append('loch ness')
vec.append((vec[word.index('scuba')] + vec[word.index('diver')]) / 2)
word.append('scuba diver')
# end hacks

vec = np.stack(vec)
word_to_idx = {w: idx for idx, w in enumerate(word)}
def word_to_vec(w):
    return vec[word_to_idx[w]]

vec /= np.linalg.norm(vec, axis=-1)[:, None]
# vec = PCA(n_components=128).fit_transform(vec)
len(word), vec.shape, f'{vec.size * vec.itemsize // 10**6} MB'

(26660, (26660, 300), '31 MB')

In [7]:
%%time
ANALOGIES = [
    ('soft', 'softer', 'hard'),
    ('airplane', 'air', 'boat'),
    ('woman', 'man', 'queen'),
    ('bedroom', 'bed', 'restroom'),
    ('big', 'small', 'long'),
    ('god', 'angel', 'devil'),
    ('computer', 'keyboard', 'car'),
    ('employee', 'boss', 'soldier'),
    ('int', 'float', 'integer'),
    # unit tests for hacky words:
    ('new york', 'usa', 'paris'),
    ('loch ness', 'uk', 'salem'),
    ('scuba diver', 'water', 'astronaut'),
]

def derived_from(word, sources):
    for src in sources:
        if word == src or word == src + 's':
            return True
    return False

for srcfrom, srcto, dstfrom in ANALOGIES:
    targetvec = word_to_vec(srcto) - word_to_vec(srcfrom) + word_to_vec(dstfrom) 
    dist = (1 - (vec * targetvec[None, :])).sum(-1)
    candidates = [word[idx] for idx in dist.argsort()[:100]]
    candidates = [w for w in candidates if not derived_from(w, [srcfrom, srcto, dstfrom])][:5]

    print(f'{srcfrom} -> {srcto} is like {dstfrom} -> {candidates[0]} [{", ".join(candidates[1:])}]')

soft -> softer is like hard -> harder [tougher, easier, faster, better]
airplane -> air is like boat -> water [sea, ocean, river, pump]
woman -> man is like queen -> king [prince, kings, royal, crown]
bedroom -> bed is like restroom -> toilet [toilets, shower, wheelchair, hygiene]
big -> small is like long -> short [longer, relatively, large, few]
god -> angel is like devil -> demon [witch, kitty, raven, ghost]
computer -> keyboard is like car -> truck [pedal, vehicle, suv, guitar]
employee -> boss is like soldier -> hero [boy, man, warrior, army]
int -> float is like integer -> floating [decimal, arithmetic, numeric, binary]
new york -> usa is like paris -> france [belgium, europe, switzerland, germany]
loch ness -> uk is like salem -> usa [united, hampshire, massachusetts, london]
scuba diver -> water is like astronaut -> nasa [space, earth, milk, moon]
CPU times: user 200 ms, sys: 47.9 ms, total: 248 ms
Wall time: 245 ms


In [8]:
glove_words = set(word)
for w in codenames_words:
    if w not in glove_words:
        print(w)

In [9]:
%%time
def get_distances(targetvec):
    return (1 - (vec * targetvec[None, :]).sum(-1))
EXAMPLES = [
    {
        'good': ["church", "cat", "atlantis"],
        'bad': ["eye", "aztec", "buck", "pin", "hospital"],
        'fail': ["fair"],
    },
    {
        'good': ["iron", "ham", "beijing"],
        'bad': ["witch", "note", "cat", "bear", "ambulance"],
        'fail': ["fall"],
    },
    {
        'good': ["robin", "screen", "server"],
        'bad': ["pole", "plate", "ground", "pupil", "iron"],
        'fail': ["novel"],
    }
]

for ex in EXAMPLES:
    good_distances = np.concatenate([get_distances(word_to_vec(w))[:, None] for w in ex['good']], -1)
    bad_distances = np.concatenate([get_distances(word_to_vec(w))[:, None] for w in ex['bad']], -1)
    fail_distances = np.concatenate([get_distances(word_to_vec(w))[:, None] for w in ex['fail']], -1)
    
    risk = 1
    num_guessed = 3
    bad_score = np.minimum(fail_distances[:, 0], np.sort(bad_distances, -1)[:, risk])
    print(bad_score[:5])
    good_score = -np.sort(good_distances, -1)[:, num_guessed - 1]
#     print(good_score[:5])
    print(np.sort(good_distances, -1)[:3])

    score = bad_score + good_score
    candidates = [word[idx] for idx in (-score).argsort()[:1000]][:10]
    print('max score', score.max())
    print('good:', ex['good'])
    print('bad:', ex['bad'])
    print('fail:', ex['fail'])
    print()
    for c in candidates:
        print(c, score[word_to_idx[c]])
    print()
    print('---')
    print()

[0.47582853 0.5133306  0.54044765 0.5138103  0.5261601 ]
[[0.51287854 0.58915204 0.69899464]
 [0.56543154 0.59968007 0.7987106 ]
 [0.5481349  0.60924804 0.7807704 ]]
max score 0.06720805
good: ['church', 'cat', 'atlantis']
bad: ['eye', 'aztec', 'buck', 'pin', 'hospital']
fail: ['fair']

ark 0.06720805
lazarus 0.0562582
aston 0.05493933
jacuzzi 0.05282885
interlude 0.05101335
skydiving 0.048659146
sauna 0.048546195
soundtracks 0.048202276
reactor 0.048033893
cayman 0.047184587

---

[0.42217493 0.5122403  0.44222897 0.48202395 0.48491848]
[[0.5619298  0.66426396 0.7364072 ]
 [0.60418576 0.6906879  0.69164383]
 [0.62309116 0.6565501  0.7291641 ]]
max score 0.11440015
good: ['iron', 'ham', 'beijing']
bad: ['witch', 'note', 'cat', 'bear', 'ambulance']
fail: ['fall']

wok 0.11440015
tong 0.10378122
nonstick 0.086284816
moroccan 0.057478428
ge 0.053643763
btu 0.048133075
gloucester 0.045795977
porto 0.045105636
isi 0.04210043
dynamo 0.039309084

---

[0.5562968  0.57261956 0.5995827  0.56116

In [10]:
b = vec.flatten()#np.array(list(range(1000)), dtype=np.float32).tobytes()
print(len(b))
with open("vec.bytes", 'wb') as f:
    f.write(b)

7998000


In [11]:
import json

with open("word.json", 'wt') as f:
    json.dump(word, f)